<a href="https://colab.research.google.com/github/ethandavenport/Optimization-I-Project-2/blob/main/project2_q7.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install gurobipy

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.3/14.3 MB 33.9 MB/s eta 0:00:00


In [3]:
import gurobipy as gp
from gurobipy import GRB
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [4]:
url = "https://raw.githubusercontent.com/ethandavenport/Optimization-I-Project-2/main/roi_company1.csv"
df_roi = pd.read_csv(url)

url = "https://raw.githubusercontent.com/ethandavenport/Optimization-I-Project-2/main/roi_company2.csv"
df_roi2 = pd.read_csv(url)

url = "https://raw.githubusercontent.com/ethandavenport/Optimization-I-Project-2/main/roi_monthly.csv"
df_month = pd.read_csv(url)

url = "https://raw.githubusercontent.com/ethandavenport/Optimization-I-Project-2/main/min_amount.csv"
df_min = pd.read_csv(url)

In [5]:
df_roi.iloc[:5, ]

,Platform,Tier,LowerBound,UpperBound,ROI
0,Print,1,0.0,0.7,0.0345
1,Print,2,0.7,2.7,0.0305
2,Print,3,2.7,4.7,0.0276
3,Print,4,4.7,inf,0.0234
4,TV,1,0.0,4.3,0.0608


In [6]:
df_month.iloc[12:17, ]

,Month,Platform,Tier,LowerBoundM,UpperBoundM,ROI
12,Jan,Facebook,2,4.2,inf,0.0531
13,Jan,LinkedIn,1,0.0,0.4,0.0487
14,Jan,LinkedIn,2,0.4,2.6,0.0559
15,Jan,LinkedIn,3,2.6,inf,0.0564
16,Jan,Instagram,1,0.0,1.2,0.0139


In [7]:
df_min.iloc[:5, ]

,Platform,MinInvestment
0,Print,0.3
1,TV,0.3
2,SEO,0.6
3,AdWords,0.8
4,Facebook,0.4


Making sure this code matches up with the solution to the linear problem

In [8]:
# Replace 'inf' with 3.0 cap (per-medium max investment)
df_roi["UpperBound"] = df_roi["UpperBound"].replace("inf", np.inf)
df_roi["UpperBound"] = df_roi["UpperBound"].astype(float)
df_roi.loc[df_roi["UpperBound"] == np.inf, "UpperBound"] = 3.0

# Now compute finite widths
df_roi["Width"] = df_roi["UpperBound"] - df_roi["LowerBound"]

# Clean up 'inf' and ensure numeric types
df_roi["UpperBound"] = df_roi["UpperBound"].replace("inf", np.inf)
df_roi["Tier"] = df_roi["Tier"].astype(int)

# Compute width per tier (investment range)
df_roi["Width"] = df_roi["UpperBound"] - df_roi["LowerBound"]

# Platform/tier dictionaries
roi = df_roi.set_index(["Platform", "Tier"])["ROI"].to_dict()
width = df_roi.set_index(["Platform", "Tier"])["Width"].to_dict()
min_invest = df_min.set_index("Platform")["MinInvestment"].to_dict()

platforms = df_roi["Platform"].unique().tolist()
budget = 10  # total $10M

# --- Build model ---
m = gp.Model("Marketing_Optimization")

# Create decision variables only for existing tiers
x = {}
z = {}
for (p, t), _ in roi.items():
    x[p, t] = m.addVar(lb=0, name=f"x_{p}_{t}")
    z[p, t] = m.addVar(vtype=GRB.BINARY, name=f"z_{p}_{t}")

# Platform-level binary variable
y = {p: m.addVar(vtype=GRB.BINARY, name=f"y_{p}") for p in platforms}

# --- Objective ---
m.setObjective(
    gp.quicksum(roi[p, t] * x[p, t] for (p, t) in roi),
    GRB.MAXIMIZE
)

# --- Constraints ---

# 1. Total budget
m.addConstr(gp.quicksum(x[p, t] for (p, t) in roi) == budget, "Budget")

# 2. Tier logic (fill lower before higher)
for p in platforms:
    platform_tiers = sorted(df_roi.loc[df_roi["Platform"] == p, "Tier"])
    for t in platform_tiers:
        # can't exceed width if active
        m.addConstr(x[p, t] <= width[p, t] * z[p, t], f"Width_{p}_{t}")
        # link sequential tiers
        if t > min(platform_tiers):
            m.addConstr(x[p, t-1] >= width[p, t-1] * z[p, t], f"Seq_{p}_{t}")

# 3. Min/max per-platform
for p in platforms:
    tiers_p = [t for t in df_roi.loc[df_roi["Platform"] == p, "Tier"]]
    total_p = gp.quicksum(x[p, t] for t in tiers_p)
    m.addConstr(total_p >= min_invest[p] * y[p], f"Min_{p}")
    m.addConstr(total_p <= 3 * y[p], f"Cap_{p}")
    if (p, 1) in z:
        m.addConstr(z[p, 1] <= y[p], f"Activate_{p}")

# 4. Managerial constraints
def total(p):
    if p not in platforms:
        return 0
    tiers_p = [t for t in df_roi.loc[df_roi["Platform"] == p, "Tier"]]
    return gp.quicksum(x[p, t] for t in tiers_p)

m.addConstr(total("Print") + total("TV") <= total("Facebook") + total("Email"), "PrintTV")
social = total("Facebook") + total("LinkedIn") + total("Instagram") + total("Snapchat") + total("Twitter")
search = total("SEO") + total("AdWords")
m.addConstr(social >= 2 * search, "SocialVsSearch")

# --- Solve ---
m.Params.OutputFlag = 0 # tell gurobi to shut up!!
m.optimize()

# --- Output ---
if m.status == GRB.OPTIMAL:
    print(f"\nOptimal total return: {m.objVal:.4f}\n")
    results = []
    for p in platforms:
        tiers_p = [t for t in df_roi.loc[df_roi["Platform"] == p, "Tier"]]
        invest = sum(x[p, t].x for t in tiers_p)
        active_tiers = [t for t in tiers_p if z[p, t].x > 0.5]
        results.append((p, invest, active_tiers))
    df_out = pd.DataFrame(results, columns=["Platform", "TotalInvestment", "ActiveTiers"])
    print(df_out.sort_values("TotalInvestment", ascending=False))


Restricted license - for non-production use only - expires 2026-11-23

Optimal total return: 0.5436

    Platform  TotalInvestment ActiveTiers
1         TV              3.0         [1]
9      Email              3.0         [1]
6  Instagram              3.0      [1, 2]
3    AdWords              1.0         [1]
0      Print              0.0          []
2        SEO              0.0          []
5   LinkedIn              0.0          []
4   Facebook              0.0          []
7   Snapchat              0.0          []
8    Twitter              0.0          []


Applying that to the monthly re-allocation problem

In [9]:

# Clean 'inf' and numeric types
df_month["UpperBoundM"] = df_month["UpperBoundM"].replace("inf", np.inf)
df_month["UpperBoundM"] = df_month["UpperBoundM"].astype(float)
df_month.loc[df_month["UpperBoundM"] == np.inf, "UpperBoundM"] = 10.0  # cap per-platform
df_month["Tier"] = df_month["Tier"].astype(int)
df_month["Width"] = df_month["UpperBoundM"] - df_month["LowerBoundM"]

months = df_month["Month"].unique().tolist()
budget = 10.0

monthly_results = []
monthly_allocations = []

# List of all platforms (for consistent columns)
all_platforms = df_month["Platform"].unique().tolist()
all_tiers = [1,2,3,4]  # maximum number of tiers

for month in months:
    print(f"\n===== {month} =====")
    df_m = df_month[df_month["Month"] == month].copy()
    platforms = df_m["Platform"].unique().tolist()
    roi = df_m.set_index(["Platform", "Tier"])["ROI"].to_dict()
    width = df_m.set_index(["Platform", "Tier"])["Width"].to_dict()

    # --- Build model ---
    m = gp.Model(f"Marketing_{month}")
    x = {}
    z = {}

    for (p, t), _ in roi.items():
        x[p, t] = m.addVar(lb=0, name=f"x_{p}_{t}")
        z[p, t] = m.addVar(vtype=GRB.BINARY, name=f"z_{p}_{t}")

    # Objective
    m.setObjective(gp.quicksum(roi[p, t] * x[p, t] for (p, t) in roi), GRB.MAXIMIZE)

    # Budget
    m.addConstr(gp.quicksum(x[p, t] for (p, t) in roi) == budget, "Budget")

    # Tier constraints
    for p in platforms:
      platform_tiers = sorted(df_m.loc[df_m["Platform"] == p, "Tier"])
      for i, t in enumerate(platform_tiers):
          # Tier t cannot exceed its width if activated
          m.addConstr(x[p, t] <= width[p, t] * z[p, t], f"Width_{p}_{t}")

          # Sequential tier activation: previous tier must be fully invested
          if i > 0:  # tier index > 0 means t>1
              t_prev = platform_tiers[i-1]
              m.addConstr(x[p, t_prev] >= width[p, t_prev] * z[p, t], f"Seq_{p}_{t}")

      # Platform cap <= 3M
      m.addConstr(gp.quicksum(x[p, t] for t in platform_tiers) <= 3, f"Cap_{p}")

      # --- NEW: enforce z=0 if total investment = 0 ---
      m.addConstr(gp.quicksum(x[p, t] for t in platform_tiers) <= 3 * gp.quicksum(z[p, t] for t in platform_tiers),
                f"ZeroInvest_{p}")

    # Managerial constraints
    def total(p):
        if p not in platforms:
            return 0
        tiers_p = [t for t in df_m.loc[df_m["Platform"] == p, "Tier"]]
        return gp.quicksum(x[p, t] for t in tiers_p)

    m.addConstr(total("Print") + total("TV") <= total("Facebook") + total("Email"), "PrintTV")
    social = total("Facebook") + total("LinkedIn") + total("Instagram") + total("Snapchat") + total("Twitter")
    search = total("SEO") + total("AdWords")
    m.addConstr(social >= 2 * search, "SocialVsSearch")

    # Solve
    m.setParam("OutputFlag", 0)
    m.optimize()

    if m.status == GRB.OPTIMAL:
        total_return = m.objVal
        roi_percent = total_return / budget
        new_budget = budget * (1 + 0.5 * roi_percent)

        print(f"Budget: ${budget:.4f}M | Return: ${total_return:.6f}M | ROI: {roi_percent*100:.2f}% | Next budget: ${new_budget:.4f}M")

        # --- Continuous investment matrix ---
        x_matrix = pd.DataFrame(index=all_platforms, columns=all_tiers)
        for p in all_platforms:
            for t in all_tiers:
                if (p, t) in x:
                    x_matrix.at[p, t] = x[p, t].x
                else:
                    x_matrix.at[p, t] = 0.0
        print("\nContinuous Investment Matrix (x):")
        print(x_matrix.round(4))

        # --- Binary decision matrix ---
        z_matrix = pd.DataFrame(index=all_platforms, columns=all_tiers)
        for p in all_platforms:
            for t in all_tiers:
                if (p, t) in z:
                    z_matrix.at[p, t] = z[p, t].x
                else:
                    z_matrix.at[p, t] = 0
        print("\nBinary Decision Matrix (z):")
        print(z_matrix.round(0))

        # Record per-platform allocation
        for p in all_platforms:
            tiers_p = df_m.loc[df_m["Platform"] == p, "Tier"].tolist()
            invest = sum(x[p, t].x for t in tiers_p) if p in platforms else 0.0
            monthly_allocations.append({
                "Month": month,
                "Platform": p,
                "Investment": invest
            })

        monthly_results.append({
            "Month": month,
            "Budget(M$)": budget,
            "Return(M$)": total_return
        })

        # Update budget for next month
        budget = new_budget

# --- Create final table ---
df_alloc = pd.DataFrame(monthly_allocations)
df_summary = pd.DataFrame(monthly_results)
df_alloc_pivot = df_alloc.pivot(index="Month", columns="Platform", values="Investment").fillna(0)
df_final = df_summary.merge(df_alloc_pivot, left_on="Month", right_index=True)

# Sort columns nicely
cols_order = ["Month", "Budget(M$)", "Return(M$)"] + sorted(all_platforms)
df_final = df_final[cols_order].round(4)

print("\n===== FINAL ALLOCATION TABLE =====")
df_final



===== Jan =====
Budget: $10.0000M | Return: $0.539420M | ROI: 5.39% | Next budget: $10.2697M

Continuous Investment Matrix (x):
             1    2    3    4
Print      3.0  0.0  0.0  0.0
TV         0.0  0.0  0.0  0.0
SEO        0.0  0.0  0.0  0.0
AdWords    0.0  0.0  0.0  0.0
Facebook   3.0  0.0  0.0  0.0
LinkedIn   0.4  2.2  0.4  0.0
Instagram  0.0  0.0  0.0  0.0
Snapchat   0.0  0.0  0.0  0.0
Twitter    1.0  0.0  0.0  0.0
Email      0.0  0.0  0.0  0.0

Binary Decision Matrix (z):
             1    2    3    4
Print      1.0  0.0    0    0
TV         1.0  0.0    0    0
SEO        1.0  0.0  0.0  0.0
AdWords    1.0  0.0  0.0    0
Facebook   1.0  0.0    0    0
LinkedIn   1.0  1.0  1.0    0
Instagram  1.0  0.0  0.0    0
Snapchat   1.0  0.0  0.0  0.0
Twitter    1.0  0.0  0.0    0
Email      1.0  0.0  0.0  0.0

===== Feb =====
Budget: $10.2697M | Return: $0.420890M | ROI: 4.10% | Next budget: $10.4802M

Continuous Investment Matrix (x):
                 1        2    3    4
Print          

,Month,Budget(M$),Return(M$),AdWords,Email,Facebook,Instagram,LinkedIn,Print,SEO,Snapchat,TV,Twitter
0,Jan,10.0000,0.5394,0.0000,0.0000,3.0000,0.0000,3.0,3.0000,0.0,0.0000,0.0,1.0000
1,Feb,10.2697,0.4209,2.6566,0.0000,3.0000,0.0000,2.1,2.3000,0.0,0.2131,0.0,0.0000
2,Mar,10.4802,0.5488,2.6960,0.0000,2.3921,0.0000,3.0,2.3921,0.0,0.0000,0.0,0.0000
3,Apr,10.7545,0.5237,1.7545,0.0000,3.0000,0.0000,3.0,3.0000,0.0,0.0000,0.0,0.0000
4,May,11.0164,0.6315,0.0000,3.0000,0.0000,2.0164,3.0,3.0000,0.0,0.0000,0.0,0.0000
5,Jun,11.3322,0.5305,0.0000,0.0000,3.0000,0.0000,3.0,3.0000,0.0,0.0000,0.0,2.3322
6,Jul,11.5974,0.6114,0.0000,0.0000,3.0000,0.0000,3.0,2.5974,3.0,0.0000,0.0,0.0000
7,Aug,11.9031,0.6167,3.0000,0.0000,0.6000,0.0000,3.0,0.6000,0.0,1.7031,0.0,3.0000
8,Sep,12.2115,0.6519,2.3705,2.1000,0.9000,0.8410,3.0,3.0000,0.0,0.0000,0.0,0.0000
9,Oct,12.5374,0.7483,0.0000,0.5374,3.0000,0.0000,3.0,3.0000,0.0,0.0000,0.0,3.0000


In [10]:
3*.0512 + 0.4*0.0487 + 2.2*0.0559 + 0.4*0.0564 + 3*0.0571 + 1*0.0496

0.53952

In [11]:
df_month[df_month['Month'] == "Jan"].sort_values(by='ROI', ascending=False)

,Month,Platform,Tier,LowerBoundM,UpperBoundM,ROI,Width
1,Jan,Print,2,6.0,10.0,0.0593,4.0
0,Jan,Print,1,0.0,6.0,0.0571,6.0
15,Jan,LinkedIn,3,2.6,10.0,0.0564,7.4
14,Jan,LinkedIn,2,0.4,2.6,0.0559,2.2
25,Jan,Twitter,3,3.7,10.0,0.0541,6.3
12,Jan,Facebook,2,4.2,10.0,0.0531,5.8
11,Jan,Facebook,1,0.0,4.2,0.0512,4.2
24,Jan,Twitter,2,2.4,3.7,0.0498,1.3
23,Jan,Twitter,1,0.0,2.4,0.0495,2.4
13,Jan,LinkedIn,1,0.0,0.4,0.0487,0.4
